In [110]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import pickle


def plot_confusion_matrix(y_true, y_pred, label,
                          cmap=plt.cm.Blues):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data

    print(cm)

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=label, yticklabels=label,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


import pandas as pd

In [133]:
c = [1, 10, 100, 1000]
gamma = [1e-3, 1e-4,.04]
features = ['Date', 'Payrolls_3mo_vs_12mo', 'Effective_Fed_Funds_12_chg', 'CPI_All_Items_3_mo_annualised',
            '10Y_Treasury_Rate_12_chg', '3M_10Y_Treasury_Spread', 'S&P_500_Index_12_chg','Recession_in_6mo','Recession_in_12mo','Recession_in_24mo']
df = pd.read_csv('/Users/sebin/Desktop/companyandngo/Data/final_features.csv')[::-1]
predicted_probability=pd.DataFrame()
past_predict = pd.DataFrame()
test_prob = pd.DataFrame()
recession_data = df[features]
#df_features.join(pd.get_dummies(df['Recession']))
output_feature =['Recession_in_6mo','Recession_in_12mo']

In [112]:
x_training_data=recession_data[recession_data['Date'] <= '2018-02-01']
x_testing_data = recession_data[recession_data['Date'] >= '2018-02-01']


In [65]:
c

array([  10.        ,   31.6227766 ,  100.        ,  316.22776602,
       1000.        ])

In [113]:
X_train, X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [228]:
x_testing_data['Recession_in_24mo']

24    0
23    0
22    0
21    0
20    0
19    0
18    0
17    0
16    0
15    0
14    0
13    0
12    0
11    0
10    0
9     0
8     0
7     0
6     0
5     0
4     0
3     0
2     0
1     0
0     0
Name: Recession_in_24mo, dtype: int64

In [140]:
for i in output_feature:
    svm_ac = {}
    for x, y in [(x, y) for x in c for y in gamma]:
        k = x, y
        # initalize the'model' and passing the SVM Classifier with respect to C and gamma value
        model = svm.SVC(C=x, kernel='rbf', gamma=y, probability=True,
                        tol=1e-3,
                        class_weight='balanced')
        accuracy = cross_val_score(model, X_train.iloc[:,1:6], y_train[i], cv=10, scoring='accuracy')
        svm_ac[k] = np.mean(accuracy)
        print("\n{:20} {:25}".format('Parameter', 'Average accuracy'))
        print('{}\t{:25}'.format(k, np.mean(accuracy)))
    print(f"\n----{i}------")
    acc_score = max(svm_ac, key=svm_ac.get)
    print(f"The best tuning parameter{acc_score}")
    svc_model = svm.SVC(C=acc_score[0], kernel='rbf', gamma=acc_score[1], probability=True,
                        tol=1e-3, random_state=123,
                        class_weight='balanced')
   
    # fitting the model with training data
    svc_model.fit(X_train.iloc[:,1:6], y_train[i])
    # predicting the model with the test data
    y_predict = svc_model.predict(X_test.iloc[:,1:6])

    # evaluating the accuracy with the ground truth and predicted value
    model_acc = accuracy_score(y_test[i], y_predict)

    print("Model Accuracy is: {}".format(model_acc))
    # classification_report((y_true, y_pred)
    predicted_probability[i+"_probability"] = svc_model.predict_proba(x_testing_data.iloc[:,1:6])[:,1]
    past_predict[i+"_probability"]=svc_model.predict_proba(X_train.iloc[:,1:6])[:,1]
    test_prob[i+"_probability"]=svc_model.predict_proba(X_test.iloc[:,1:6])[:,1]
    filename = i+'.sv'
    pickle.dump(svc_model, open(filename, 'wb'))
      


Parameter            Average accuracy         
(1, 0.001)	       0.7359941944847604

Parameter            Average accuracy         
(1, 0.0001)	      0.44034833091436865

Parameter            Average accuracy         
(1, 0.04)	       0.6767779390420899

Parameter            Average accuracy         
(10, 0.001)	        0.699854862119013

Parameter            Average accuracy         
(10, 0.0001)	       0.7359941944847604

Parameter            Average accuracy         
(10, 0.04)	       0.6520682148040639

Parameter            Average accuracy         
(100, 0.001)	       0.6748911465892597

Parameter            Average accuracy         
(100, 0.0001)	        0.699854862119013

Parameter            Average accuracy         
(100, 0.04)	       0.6750362844702467

Parameter            Average accuracy         
(1000, 0.001)	       0.6768867924528302

Parameter            Average accuracy         
(1000, 0.0001)	       0.6748911465892597

Parameter            Average accuracy         
(

In [135]:

predicted_probability['Date']= x_testing_data['Date'].tolist()

In [136]:
predicted_probability

,Recession_in_6mo_probability,Recession_in_12mo_probability,Date
0,0.131479,0.135684,2018-02-01
1,0.152996,0.145310,2018-03-01
2,0.157983,0.183644,2018-04-01
3,0.156739,0.201196,2018-05-01
4,0.174656,0.248546,2018-06-01
5,0.188151,0.221744,2018-07-01
6,0.200872,0.262821,2018-08-01
7,0.199036,0.291446,2018-09-01
8,0.192339,0.304564,2018-10-01
9,0.212307,0.333243,2018-11-01


In [137]:
past_predict['Date']= X_train['Date'].tolist()
test_prob['Date']=X_test['Date'].tolist()


In [40]:
past_predict.append(test_prob)

,Recession_in_6mo_probability,Recession_in_12mo_probability,Recession_in_24mo_probability,Date
0,0.352579,0.330135,0.156131,1989-12-01
1,0.095807,0.310598,0.156326,1955-12-01
2,0.145612,0.144153,0.142512,1963-06-01
3,0.269259,0.322594,0.156075,1965-04-01
4,0.121888,0.153016,0.143826,1995-09-01
...,...,...,...,...
221,0.297531,0.335170,0.156713,1989-08-01
222,0.011730,0.005666,0.104341,2004-06-01
223,0.087227,0.079532,0.134358,1961-11-01
224,0.079628,0.093648,0.136780,1977-10-01


In [194]:
past_predict.append(predicted_probability)

,Recession_in_6mo_probability,Recession_in_12mo_probability,Recession_in_24mo_probability,Date
0,0.029583,0.014172,0.105463,1991-06-01
1,0.057241,0.028325,0.104833,2015-08-01
2,0.165625,0.159222,0.103868,1973-02-01
3,0.210281,0.196792,0.103727,1956-07-01
4,0.012034,0.004622,0.106102,1985-04-01
...,...,...,...,...
20,0.387151,0.170527,0.103286,2019-10-01
21,0.335060,0.144865,0.103475,2019-11-01
22,0.320268,0.154348,0.103512,2019-12-01
23,0.343812,0.164299,0.103453,2020-01-01


In [139]:
file=['modelRecession_in_6mo.sv','modelRecession_in_12mo.sv']

for i in file:
    svmc = pickle.load(open(i, 'rb'))
    print(svmc.predict_proba(test.iloc[:,1:6])[:,1])
    
    
    



[0.48641245]
[0.40234626]


In [131]:
test=recession_data[recession_data['Date'] == '2007-02-01']

test

,Date,Payrolls_3mo_vs_12mo,Effective_Fed_Funds_12_chg,CPI_All_Items_3_mo_annualised,10Y_Treasury_Rate_12_chg,3M_10Y_Treasury_Spread,S&P_500_Index_12_chg,Recession_in_6mo,Recession_in_12mo,Recession_in_24mo
156,2007-02-01,1.001109,1.171492,1.044813,1.032823,-0.31,1.098512,0,1,1


In [106]:
X_train,X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [109]:

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for i in output_feature:
    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='accuracy'
    )
    clf.fit(X_train.iloc[:,1:6], y_train[i])

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print(f"------{i}---------")
    print()

/Users/sebin/anaconda3/envs/p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameters set found on development set:

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.873 (+/-0.005) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.873 (+/-0.005) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.873 (+/-0.005) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.873 (+/-0.005) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.873 (+/-0.005) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.873 (+/-0.005) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.873 (+/-0.005) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.873 (+/-0.005) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.873 (+/-0.005) for {'C': 1, 'kernel': 'linear'}
0.873 (+/-0.005) for {'C': 10, 'kernel': 'linear'}
0.873 (+/-0.005) for {'C': 100, 'kernel': 'linear'}
0.873 (+/-0.005) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              

/Users/sebin/anaconda3/envs/p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameters set found on development set:

{'C': 100, 'kernel': 'linear'}

Grid scores on development set:

0.856 (+/-0.005) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.856 (+/-0.005) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.856 (+/-0.005) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.856 (+/-0.005) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.856 (+/-0.005) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.856 (+/-0.005) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.852 (+/-0.001) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.856 (+/-0.005) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.856 (+/-0.005) for {'C': 1, 'kernel': 'linear'}
0.856 (+/-0.019) for {'C': 10, 'kernel': 'linear'}
0.871 (+/-0.023) for {'C': 100, 'kernel': 'linear'}
0.871 (+/-0.023) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision  

/Users/sebin/anaconda3/envs/p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best parameters set found on development set:

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.857 (+/-0.001) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.857 (+/-0.001) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.857 (+/-0.001) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.857 (+/-0.001) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.857 (+/-0.001) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.857 (+/-0.001) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.857 (+/-0.001) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.857 (+/-0.001) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.857 (+/-0.001) for {'C': 1, 'kernel': 'linear'}
0.857 (+/-0.001) for {'C': 10, 'kernel': 'linear'}
0.857 (+/-0.001) for {'C': 100, 'kernel': 'linear'}
0.857 (+/-0.001) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              